*This notebook made by:*

**Farnaz Baksh**

**Matevz Zorec**

During our **Introduction to Data Science course** in Autumn of 2021

Course Code: *LTAT.02.002*

Project title:

>**Team B1: UT Student Profiling based on Moodle Log Data**

In [1]:
import pandas as pd
import numpy as np

from google_trans_new import google_translator
translator = google_translator()

In [10]:
translate_text = translator.translate('Sedaj stvar deluje in pocasi prevaja.',lang_tgt='en')  
print(translate_text)

Now the thing works and slowly translates. 


[get pip install dir windows](https://stackoverflow.com/questions/49028533/pip-packages-path-windows/49028561)

[fix google trans new error](https://stackoverflow.com/questions/68214591/python-google-trans-new-translate-raises-error-jsondecodeerror-extra-data)

```bash
pip install google_trans_new
```
[google_trans_new](https://pypi.org/project/google-trans-new/)

**instructions to fix:**

*copy this into terminal:*
```bash
python -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())"
```
**Change line 151 in google_trans_new/google_trans_new.py which is:**
```python
response = (decoded_line + ']') to response = decoded_line
```
Then, restart your notebook kernel and run.

**Goal 1:** identify students who may be struggling based on their early activities (from activity logs)


**Goal 2:** predict the student’ final grades in the course (from activity logs)


**Goal 3:** discover common activity patterns of the students and create profiles of typical students (from activity logs).


In [3]:
grades = pd.read_excel("02-05 Hinded.xlsx") # all files we are operating with are excel files and not csv
grades # Hinded == Grades

,Eesnimi,Perenimi,ID-number,Meiliaadress,Test:1. nädala test (Punktid),Test:2. nädala test (Punktid),Test:3. nädala test (Punktid),Test:4. nädala test (Punktid),Test:5. nädala test (Punktid),Test:6. nädala test (Punktid),...,VPL harjutus:Pokkerikäte tuvastamise automaatkontroll (Punktid),Test:Kontrolltöö harjutustest (Punktid),Test:Kontrolltöö harjutustest (Punktid).1,Test:Näidiseksami arvestuslik osa (Punktid),Test:Näidiseksami punktilise osa 1. ülesanne (Punktid),Ülesanne:Näidiseksami logide ja video esitamine (Punktid),Harjutamine kokku (Punktid),Kogutulemus (Punktid),Hinne (Punktid),Sellest kursusest viimati alla laaditud
0,Mark-Erik,Aan,50105090226,mark-erik.aan@ut.ee,0.5,0.5,0.5,0.5,0.5,0.5,...,-,8,9,4,9.6,NaN,-,109.40,100.00,1612512348
1,Sass Albert,Aare,50106202741,sass.albert.aare@ut.ee,0.48,0.46,0.5,0.45,0.45,0.45,...,-,8,6.5,-,-,NaN,-,97.60,97.60,1612512348
2,Mia-Nicolena,Aaver,49711072726,mia.aaver@ut.ee,0.5,0.5,0.5,0.5,0.5,0.5,...,-,8,8.75,4,9.6,NaN,-,95.68,95.68,1612512348
3,Heili,Aavola,60109126818,heili.aavola@ut.ee,0.5,0.45,-,0.5,0.5,0.46,...,-,8,6.5,3,10,NaN,-,73.13,73.13,1612512348
4,Arash,Adab Avazeh,38603270069,arash.adab.avazeh@ut.ee,0.48,0.38,0.35,0.23,0.31,0.36,...,-,6,6.3,2.1,-,NaN,-,42.23,0.10,1612512348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,Joonas,Väli,50003092774,joonas.vali@ut.ee,0.5,0.5,0.46,0.48,0.41,0.39,...,-,-,-,4,-,NaN,-,98.81,98.81,1612512355
338,Julius,Välja,50011131419,julius.valja@ut.ee,0.46,0.5,0.5,0.48,0.5,0.5,...,-,8,-,4,8.7,NaN,-,104.90,100.00,1612512355
339,Ott-Kaarel,Vään,50102182738,ott-kaarel.vaan@ut.ee,-,0.49,0.48,-,-,-,...,-,-,-,3,-,NaN,-,81.07,81.07,1612512355
340,Jane,Õunaid,60105140013,jane.ounaid@ut.ee,-,0.5,0.5,0.5,0.35,0.49,...,-,8,8.75,3.5,6.1,NaN,-,85.59,85.59,1612512355


In [4]:
logbook_Uld_02 = pd.read_excel("logs_LTAT.03.001_20210207-0948_Üld_02.xlsx") # naming convention: logbook_Uld_02
logbook_Uld_02 = logbook_Uld_02.head(100) # Uld_02 == lab group Uld 02
# decided to store just the 1st 100 values for now to make processing easier...
logbook_Uld_02

,Aeg,Kasutaja täisnimi,Mõjutatud kasutaja,Sündmuse kontekst,Komponent,Sündmuse nimi,Kirjeldus,Päritolu,IP-aadress
0,5.02.2021 18:31:29,Emma Anni Koppel,-,Kursus: Programmeerimine (LTAT.03.001),Süsteem,Kursust on vaadatud.,The user with id '131421' viewed the course wi...,web,85.253.216.96
1,5.02.2021 18:31:24,Emma Anni Koppel,Emma Anni Koppel,Kursus: Programmeerimine (LTAT.03.001),Õppijaraport,Hinnete kasutajaaruannet on vaadatud,The user with id '131421' viewed the user repo...,web,85.253.216.96
2,5.02.2021 18:31:13,Emma Anni Koppel,-,Kursus: Programmeerimine (LTAT.03.001),Süsteem,Kursust on vaadatud.,The user with id '131421' viewed the course wi...,web,85.253.216.96
3,5.02.2021 14:37:22,Heidi Meier,-,Kursus: Programmeerimine (LTAT.03.001),Süsteem,Kursust on vaadatud.,The user with id '63854' viewed the course wit...,web,2001:7d0:88db:a980:2c92:2c1a:de5e:b781
4,5.02.2021 13:58:00,Heidi Meier,-,Leht: Videote lisamaterjal,Leht,Kursusemoodulit on vaadatud.,The user with id '63854' viewed the 'page' act...,web,2001:7d0:88db:a980:2c92:2c1a:de5e:b781
...,...,...,...,...,...,...,...,...,...
95,27.01.2021 18:32:03,Heidi Meier,-,Ülesanne: Projekti lõppversioon,Ülesanne,Hindamistabelit on vaadatud,The user with id '63854' viewed the grading ta...,web,2001:7d0:88db:a980:fc06:79db:b0bf:fd0f
96,27.01.2021 18:32:02,Heidi Meier,-,Ülesanne: Projekti lõppversioon,Ülesanne,Kursusemoodulit on vaadatud.,The user with id '63854' viewed the 'assign' a...,web,2001:7d0:88db:a980:fc06:79db:b0bf:fd0f
97,27.01.2021 18:32:00,Heidi Meier,-,Ülesanne: Projekti lõppversioon,Ülesanne,Esitatud töö olekut on vaadatud.,The user with id '63854' has viewed the submis...,web,2001:7d0:88db:a980:fc06:79db:b0bf:fd0f
98,27.01.2021 18:32:00,Heidi Meier,-,Ülesanne: Projekti lõppversioon,Ülesanne,Kursusemoodulit on vaadatud.,The user with id '63854' viewed the 'assign' a...,web,2001:7d0:88db:a980:fc06:79db:b0bf:fd0f


Loading the above logbook in took roughly 13 seconds... It is possible that loading in all log files at once would work but take up to 5 minutes.

Naming convention for log dataframes.

>logbook_Uld_02

The appendix of "logbook" shall be the log file of that particular lab group.

Therefore we must understand the overall course structure very well.


Grades dataframe contains the complete gradebook from the course.

---

The first point to mention here is that all column descriptors are in Estonian.

We do not speak Estonian, therefore we will rename all columns with appropriate descriptors.

A descriptor set was provided to us but it did not contain concise translations but instead relevant explanations for what each column represents.

---

The gradebook contains the following columns is of the following shape:

In [5]:
print(grades.columns) # .columns returns all of the column names in an iterable list
print(grades.shape) # pandas native .shape method returns the shape of the overall dataframe class

print(logbook_Uld_02.columns) # .columns returns all of the column names in an iterable list
print(logbook_Uld_02.shape) # pandas native .shape method returns the shape of the overall dataframe class
logbook_Uld_02['Sündmuse kontekst'].value_counts()

Index(['Eesnimi', 'Perenimi', 'ID-number', 'Meiliaadress',
       'Test:1. nädala test (Punktid)', 'Test:2. nädala test (Punktid)',
       'Test:3. nädala test (Punktid)', 'Test:4. nädala test (Punktid)',
       'Test:5. nädala test (Punktid)', 'Test:6. nädala test (Punktid)',
       ...
       'VPL harjutus:Pokkerikäte tuvastamise automaatkontroll (Punktid)',
       'Test:Kontrolltöö harjutustest (Punktid)',
       'Test:Kontrolltöö harjutustest (Punktid).1',
       'Test:Näidiseksami arvestuslik osa (Punktid)',
       'Test:Näidiseksami punktilise osa 1. ülesanne (Punktid)',
       'Ülesanne:Näidiseksami logide ja video esitamine (Punktid)',
       'Harjutamine kokku (Punktid)', 'Kogutulemus (Punktid)',
       'Hinne (Punktid)', 'Sellest kursusest viimati alla laaditud'],
      dtype='object', length=111)
(342, 111)
Index(['Aeg', 'Kasutaja täisnimi', 'Mõjutatud kasutaja', 'Sündmuse kontekst',
       'Komponent', 'Sündmuse nimi', 'Kirjeldus', 'Päritolu', 'IP-aadress'],
      dtype='ob

Kursus: Programmeerimine (LTAT.03.001)                                   66
Ülesanne: Projekti lõppversioon                                          20
VPL harjutus: 27.01 eksami punktilise osa 2. ja 3. ülesande esitamine     7
Leht: Videote lisamaterjal                                                4
Leht: Programmeerimise õppimise hea tava                                  1
URL: Kursuse korraldus ja hindamine                                       1
URL: Video - Regulaaravaldised (Loengusalvestis)                          1
Name: Sündmuse kontekst, dtype: int64

---

Each of the 342 rows contains the entire gradebook of one student. Each row represents that one students entire grade performance during this one semester.

This gradebook for each student features one hundred and eleven columns.

---

In [7]:
# TODO: translate/rename everything in Estonian to English to reduce cogn load...

logbook_Uld_02_translated = pd.DataFrame(columns=['Time',
                                                  'User full name',
                                                  'Affected user',
                                                  'Event context',
                                                  'Component',
                                                  'Event Name',
                                                  'Description',
                                                  'Origin',
                                                  'IP Address'])


#logbook_Uld_02_translated['Context of the event'] = logbook_Uld_02['Sündmuse kontekst'].apply(translator.translate, src='et', dest='en').apply(getattr, args=('text',))
#logbook_Uld_02_translated['Context of the event'] = logbook_Uld_02['Sündmuse kontekst'].apply(lambda x: translator.translate(x, src='et', dest='en').text)

logbook_Uld_02_translated['Time'] = logbook_Uld_02['Aeg']
logbook_Uld_02_translated['User full name'] = logbook_Uld_02['Kasutaja täisnimi']
logbook_Uld_02_translated['Affected user'] = logbook_Uld_02['Mõjutatud kasutaja']
logbook_Uld_02_translated['Event context'] = logbook_Uld_02['Sündmuse kontekst']
logbook_Uld_02_translated['Component'] = logbook_Uld_02['Komponent'].apply(lambda x: translator.translate(x, lang_src='et', lang_tgt='en'))
logbook_Uld_02_translated['Event Name'] = logbook_Uld_02['Sündmuse nimi']
logbook_Uld_02_translated['Description'] = logbook_Uld_02['Kirjeldus']
logbook_Uld_02_translated['Origin'] = logbook_Uld_02['Päritolu']
logbook_Uld_02_translated['IP Address'] = logbook_Uld_02['IP-aadress']

#.apply(lambda x: translator.translate(x, src='et', dest='en').text)
#.apply(lambda x: translator.translate(x, dest='en').text)

logbook_Uld_02_translated.head(10)


,Time,User full name,Affected user,Event context,Component,Event Name,Description,Origin,IP Address
0,5.02.2021 18:31:29,Emma Anni Koppel,-,Kursus: Programmeerimine (LTAT.03.001),System,Kursust on vaadatud.,The user with id '131421' viewed the course wi...,web,85.253.216.96
1,5.02.2021 18:31:24,Emma Anni Koppel,Emma Anni Koppel,Kursus: Programmeerimine (LTAT.03.001),Student report,Hinnete kasutajaaruannet on vaadatud,The user with id '131421' viewed the user repo...,web,85.253.216.96
2,5.02.2021 18:31:13,Emma Anni Koppel,-,Kursus: Programmeerimine (LTAT.03.001),System,Kursust on vaadatud.,The user with id '131421' viewed the course wi...,web,85.253.216.96
3,5.02.2021 14:37:22,Heidi Meier,-,Kursus: Programmeerimine (LTAT.03.001),System,Kursust on vaadatud.,The user with id '63854' viewed the course wit...,web,2001:7d0:88db:a980:2c92:2c1a:de5e:b781
4,5.02.2021 13:58:00,Heidi Meier,-,Leht: Videote lisamaterjal,Sheet,Kursusemoodulit on vaadatud.,The user with id '63854' viewed the 'page' act...,web,2001:7d0:88db:a980:2c92:2c1a:de5e:b781
5,5.02.2021 13:55:11,Heidi Meier,-,Kursus: Programmeerimine (LTAT.03.001),System,Kursust on vaadatud.,The user with id '63854' viewed the course wit...,web,2001:7d0:88db:a980:2c92:2c1a:de5e:b781
6,5.02.2021 13:52:46,Heidi Meier,-,Kursus: Programmeerimine (LTAT.03.001),System,Kursust on vaadatud.,The user with id '63854' viewed the course wit...,web,2001:7d0:88db:a980:2c92:2c1a:de5e:b781
7,5.02.2021 13:30:57,Heidi Meier,-,Leht: Videote lisamaterjal,Sheet,Kursusemoodulit on vaadatud.,The user with id '63854' viewed the 'page' act...,web,2001:7d0:88db:a980:2c92:2c1a:de5e:b781
8,5.02.2021 13:30:54,Heidi Meier,-,Kursus: Programmeerimine (LTAT.03.001),System,Kursust on vaadatud.,The user with id '63854' viewed the course wit...,web,2001:7d0:88db:a980:2c92:2c1a:de5e:b781
9,5.02.2021 13:19:00,Heidi Meier,-,Kursus: Programmeerimine (LTAT.03.001),System,Kursust on vaadatud.,The user with id '63854' viewed the course wit...,web,2001:7d0:88db:a980:2c92:2c1a:de5e:b781


[rename columns pandas(stack overflow)](https://stackoverflow.com/questions/11346283/renaming-column-names-in-pandas)

[translate dataframe pandas (stack overflow)](https://stackoverflow.com/questions/67303463/how-to-translate-other-languages-to-english-in-pandas-dataframe)

<a href="https://stackoverflow.com/questions/39415653/jupyter-notebook-open-link-in-new-tab" target="_blank">jupyter notebook open link in new tab (stack overflow)</a>

[pandas apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)

[how to fix googletrans](https://stackoverflow.com/questions/52455774/googletrans-stopped-working-with-error-nonetype-object-has-no-attribute-group) 

tl:dr = pip install pip install google_trans_new


---

For purposes of this project, we were asked not to share student information.

We are not bound by NDA or other legal documentation to do so but will gladly abide.

Therefore we shall prepare a separate csv document, featuring only the columns:

*first name, last name, ID number, email address*

We plan on relying on the ID-numbering system as it was inherently designed to anonymise key student information.

We will retain other student data in case we need it later to simplify analysis, as ID numbers are less intuitive to work with and can inherently increase cognitive load. This key file is called: **"Student_NAME_ID_link.csv"**

---


In [ ]:
# TODO: extract first names, last name, ID-number, e-mail columns and save them to a csv file: "Student_NAME_ID_link.csv"
students = grades[['Eesnimi', 'Perenimi', 'ID-number', 'Meiliaadress']].copy()
students.to_csv('students.csv')

In [ ]:
# TODO: drop the same columns except ID-number from the gradebook dataset (make a copy)
#grades.drop(['Eesnimi', 'Perenimi', 'ID-number', 'Meiliaadress'], axis=1, inplace=True)
grades
students

---

Upon analyzing the data, we saw that there are columns that do not contain information relevant to our goals. 

We can therefore first drop all columns identified to contain irrelevant information.

---

In [ ]:
# TODO: understand what each columns contains and identify if this information is relevant with regards to our goals

In [ ]:
# TODO: 